In [1]:
#데이터 불러오기
import pandas as pd
fish = pd.read_csv('data/fish.csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
pd.unique(fish['Species']) #Species열의 값 추출

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [4]:
#입력데이터 생성
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy() #Species 열은 빼고 데이터를 만든다
fish_input[:5]

array([[242.    ,  25.4   ,  30.    ,  11.52  ,   4.02  ],
       [290.    ,  26.3   ,  31.2   ,  12.48  ,   4.3056],
       [340.    ,  26.5   ,  31.1   ,  12.3778,   4.6961],
       [363.    ,  29.    ,  33.5   ,  12.73  ,   4.4555],
       [430.    ,  29.    ,  34.    ,  12.444 ,   5.134 ]])

In [5]:
#타깃데이터 생성
fish_target = fish['Species'].to_numpy()

In [9]:
#훈련데이터와 테스트 세트로 분류
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42
)

In [10]:
#훈련세트와 테스트세트 표준화(거리기반이므로 표준화 전처리)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input) #훈련세트만으로 훈련
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [12]:
#k-최근접 이웃 분류 모델 생성
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors = 3) #이웃개수 3으로 지정
kn.fit(train_scaled, train_target) #표준화된 훈련세트와 훈련타깃으로 훈련

#점수확인
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [13]:
kn.classes_ #알파벳으로 정렬되어 있음

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [14]:
kn.predict(test_scaled[:5]) #0~4까지 타깃값 예측

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

In [15]:
test_scaled[:5] #예측할 데이터

array([[-0.88741352, -0.91804565, -1.03098914, -0.90464451, -0.80762518],
       [-1.06924656, -1.50842035, -1.54345461, -1.58849582, -1.93803151],
       [-0.54401367,  0.35641402,  0.30663259, -0.8135697 , -0.65388895],
       [-0.34698097, -0.23396068, -0.22320459, -0.11905019, -0.12233464],
       [-0.68475132, -0.51509149, -0.58801052, -0.8998784 , -0.50124996]])

In [20]:
#확률 확인
import numpy as np

proba = kn.predict_proba(test_scaled[:5]) #확률확인
np.round(proba, decimals = 4) #decimals로 소수점 네번째 자리까지 표기
                              #다섯번째 자리에서 반올림한다.
                              #classes_의 타겟클래스 : array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']
                              #array의 순서에 맞춰 확률 표시

array([[0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ]])

In [21]:
#네번째 샘플의 최근접 이웃 클래스 확인
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


#### 로지스틱 회귀